<a href="https://colab.research.google.com/github/deer-in-haze/gilusis_mokymas/blob/main/Migle_Lukoseviciute_lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello world")

Hello world
